In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pathlib
data = pathlib.Path('/content/gdrive/MyDrive/Data')

In [ ]:
pip install pennylane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pennylane as qml
from pennylane import numpy as np
import torch
import numpy as np
from pennylane.templates import RandomLayers
import torch
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random

In [ ]:
pip install PennyLane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install pennylane-cirq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install qsimcirq


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 6.9 MB/s 
     |████████████████████████████████| 213 kB 41.6 MB/s 


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
dev = qml.device('cirq.qsim', wires=4)
# Random circuit parameters
rand_params = np.random.uniform(high=2 * np.pi, size=(1, 4))
@qml.qnode(dev,interface="torch")
def circuit(phi):
    
    for j in range(4):
       
     
        qml.RY(np.pi * phi[j], wires=j)
    RandomLayers(rand_params, wires=list(range(4)))



    # Measurement producing 4 classical output values
    return [qml.expval(qml.PauliZ(j)) for j in range(4)]

In [ ]:
def loop(data,bsz):
    data1=[]
    for i in range(bsz):
        d1=(data[i][0],data[i][1],data[i][2],data[i][3])
        
        res=circuit(d1).to(device)
        data1.append(res)
    res=torch.stack(data1)
    res.to(device)
    return res

In [ ]:
def new(x):
        bsz = x.shape[0]
        size = 28
        x = x.view(bsz, size, size)
        x.to(device)

        data_list = []

        for c in range(0, size, 2):
            for r in range(0, size, 2):
                data = torch.transpose(torch.cat((x[:, c, r], x[:, c, r+1], x[:, c+1, r], x[:, c+1, r+1],
                                                 
                                                
                                                 
                                                  
                                                  
                                                  
                                                  )).view(4, bsz), 0, 1)
                
                data =loop(data,bsz).to(device)
                data_list.append(data.view(bsz, 4))
        
        result = torch.cat(data_list, dim=1).float()
        result.to(device)
        
        return result

In [ ]:
class HybridModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        
        self.linear = torch.nn.Linear(4*14*14, 3).to(device)
        
    
    def forward(self, x):
        with torch.no_grad():
           
            x = new(x).to(device)
        
        x = self.linear(x).to(device)
        y=F.log_softmax(x, -1).to(device)
        return y

In [ ]:
import time
import os
import copy

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, transforms

# Pennylane
import pennylane as qml
from pennylane import numpy as np

torch.manual_seed(42)
np.random.seed(42)

# Plotting
import matplotlib.pyplot as plt

# OpenMP: number of parallel threads.
os.environ["OMP_NUM_THREADS"] = "4"

In [ ]:
n_qubits = 4                # Number of qubits
step = 0.0001               # Learning rate
batch_size=32           # Number of samples for each training step
num_epochs =6              # Number of training epochs
q_depth = 6                 # Depth of the quantum circuit (number of variational layers)
gamma_lr_scheduler = 0.1    # Learning rate reduction applied every 10 epochs.
q_delta = 0.01              # Initial spread of random quantum weights
start_time = time.time()

In [ ]:
data_transforms = {
    "train": transforms.Compose(
        [
            # transforms.RandomResizedCrop(224),     # uncomment for data augmentation
            # transforms.RandomHorizontalFlip(),     # uncomment for data augmentation
            transforms.Resize((28,28)),
            
            transforms.ToTensor(),
            # Normalize input channels using mean values and standard deviations of ImageNet.
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            transforms.Grayscale(num_output_channels=1),
        ]
    ),
    "test": transforms.Compose(
        [
            transforms.Resize((28,28)),
            
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
           transforms.Grayscale(num_output_channels=1),
        ]
    ),
}

data_dir = data
image_datasets = {
    x if x == "train" else "test": datasets.ImageFolder(
        os.path.join(data_dir, x), data_transforms[x]
    )
    for x in ["train", "test"]
}
dataset_sizes = {x: len(image_datasets[x]) for x in ["train", "test"]}
class_names = image_datasets["train"].classes

# Initialize dataloader
dataloaders = {
    x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True)
    for x in ["train", "test"]
}

# function to plot images
def imshow(inp, title=None):
    """Display image from tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    # Inverse of the initial normalization operation.
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model_hybrid = HybridModel().to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
optimizer_hybrid = optim.Adam(model_hybrid.parameters(), lr=step)

In [ ]:
exp_lr_scheduler = lr_scheduler.StepLR(
    optimizer_hybrid, step_size=10, gamma=gamma_lr_scheduler
)

In [ ]:
tloss=[]
vloss=[]
tacc=[]
vacc=[]

def train_model(model, criterion, optimizer, scheduler, num_epochs=15):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs).to(device)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            if phase == 'train':
                tloss.append(epoch_loss)
                tacc.append(epoch_acc)
            if phase == 'test':
                vloss.append(epoch_loss)
                vacc.append(epoch_acc)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model_hybrid = train_model(
    model_hybrid, criterion, optimizer_hybrid, exp_lr_scheduler, num_epochs=15
)

Epoch 0/14
----------
train Loss: 0.7509 Acc: 0.6779
test Loss: 0.5738 Acc: 0.7849

Epoch 1/14
----------
train Loss: 0.5367 Acc: 0.7998
test Loss: 0.4639 Acc: 0.8269

Epoch 2/14
----------


In [ ]:
total_params = sum(p.numel() for p in model_hybrid.parameters())
total_params

In [ ]:
learn_params = sum(p.numel() for p in model_hybrid.parameters() if p.requires_grad)
learn_params

In [ ]:
tacc1=[]
for i in range (15):
    tacc1.append(tacc[i].cpu())

In [ ]:
vacc1=[]
for i in range (15):
    vacc1.append(vacc[i].cpu())

In [ ]:

plt.plot(tacc1,'-o')
plt.plot(vacc1,'-o')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['Train','Valid'])
plt.title('Train vs Valid Accuracy')

plt.show()

In [ ]:
plt.plot(tloss,'-o')
plt.plot(vloss,'-o')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['Train','Valid'])
plt.title('Train vs Valid Loss')

plt.show()

In [ ]:
testloader = torch.utils.data.DataLoader(image_datasets['test'], batch_size=10, shuffle=True, num_workers=2)
correct, total = 0, 0
correct_count, all_count = 0, 0
for images,labels in testloader:
  for i in range(len(labels)):
    if torch.cuda.is_available():
        images = images.cuda()
        labels = labels.cuda()
    img = images[i].view(-1,1, 64, 64)
    with torch.no_grad():
        logps = model_hybrid(img)

    
    ps = torch.exp(logps)
    probab = list(ps.cpu()[0])
    pred_label = probab.index(max(probab))
    true_label = labels.cpu()[i]
    if(true_label == pred_label):
      correct_count += 1
    all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))

In [ ]:
import seaborn as sns
import pandas as pd
test_loader = torch.utils.data.DataLoader(image_datasets['test'], batch_size=10, shuffle=True, num_workers=2)
nb_classes = 3
confusion_matrix = np.zeros((nb_classes, nb_classes))
with torch.no_grad():
    for i, (inputs, classes) in enumerate(test_loader):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_hybrid(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

plt.figure(figsize=(5,5))

#class_names = list(label2class.values())
df_cm = pd.DataFrame(confusion_matrix, index=class_names, columns=class_names).astype(int)
heatmap = sns.heatmap(df_cm, annot=True, fmt="d")

heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right',fontsize=5)
heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right',fontsize=5)
plt.ylabel('True label')
plt.xlabel('Predicted label')
;